# A value history player widget

Consider a value widget `w`.
Player(`w`) wraps this widget to maintain a history of all the values that have been assigned to it.
In addition, it adds controls analogous to those of audio/video/... players to
replay that history.

In [2]:
%run HistoryPlayerWidget.py

Current bug: when playing at the end while new values are being produced, the current clock can't be cancelled
GIL issue?

In [3]:
w = Text()
p = Player(w)
p

Player(children=(Text(value=''), HBox(children=(Button(icon='fast-backward', layout=Layout(width='initial'), s…

Running command step 1
consumed new value
Running command step 1
consuming new value timed out; time: 0 / 1
consuming new value timed out; time: 0 / 1
consuming new value timed out; time: 0 / 1
consuming new value timed out; time: 0 / 1
consuming new value timed out; time: 0 / 1
canceling
step_backward
step_backward done
step_backward
step_backward done
step_backward
step_backward done


In [4]:
p.value = "9"

In [5]:
for i in range(10):
    #
    print(i)
    p.value = str(i**2)

0
1
consumed new value
2
Running command step 1
consumed new value
3
Running command step 1
consumed new value4

Running command step 1
consumed new value
5
Running command step 1
consumed new value
6
Running command step 1
consumed new value
7
Running command step 1
consumed new value
8
Running command step 1
consumed new value9

Running command step 1
consumed new value


In [8]:
p = Player(Text())
def mysort(l):
    p.value = str(l)
    for i in range(len(l)):
        for j in range(i, 0, -1):
            if l[j-1] <= l[j]:
                break
            l[j-1], l[j] = l[j], l[j-1]
            p.value = str(l)
    return l
p

Running command step 1


Player(children=(Text(value=''), HBox(children=(Button(icon='fast-backward', layout=Layout(width='initial'), s…

In [9]:
mysort([6,5,4,3,2,1])

Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command


[1, 2, 3, 4, 5, 6]

In [18]:
p._model._clock